In [ ]:
# y_seq_len + merged_future_prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import random
import os
import yfinance as yf
import timeit
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from torch.nn.modules.transformer import TransformerDecoder, TransformerDecoderLayer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def create_sequences(data, X_seq_len, y_seq_len):
    X = []
    y = []

    for i in range(len(data) - X_seq_len - y_seq_len + 1):
        X.append(data[i : i + X_seq_len])
        y.append(data[i + X_seq_len : i + X_seq_len + y_seq_len])

    return np.array(X), np.array(y)

def calculate_indicators(data, period=10, acceleration_factor=2, rsi_period=14, short_ema_period=12,
                         long_ema_period=26, signal_period=9, vol_period=20, bollinger_band_window_size=20):
    # Check if 'Close' column exists in the data
    if 'Close' in data.columns:
        # Calculate daily returns
        data['Return'] = data['Close'].pct_change()

        # Price Range
        data['Price Range'] = data['High'] - data['Low']

        # Candle Body Size
        data['Body Size'] = data['Open'] - data['Close']

        # Upper Shadow Size
        data['Upper Shadow'] = data.apply(lambda row: row.High - max(row.Open, row.Close), axis=1)

        # Lower Shadow Size
        data['Lower Shadow'] = data.apply(lambda row: min(row.Open, row.Close) - row.Low, axis=1)

        # # Directional Movement
        # data['Directional Movement'] = data.apply(lambda row: 1 if row.Close > row.Open else (-1 if row.Close < 0 else 0), axis=1)

        # Calculate Moving Averages
        # data['MA_25'] = data['Close'].rolling(window=25).mean()
        # data['MA_90'] = data['Close'].rolling(window=90).mean()
        # data['MA_200'] = data['Close'].rolling(window=200).mean()

        zlema = np.zeros(shape=(len(data),))

        # Compute the initial ZLEMA value for each row.
        for i in range(period):
          zlema[i] = data['Close'].iloc[:i + 1].mean()

        # Compute the remaining ZLEMA values for each row.
        for i in range(period, len(data)):
          zlema[i] = zlema[i - 1] + acceleration_factor * (data['Close'].iloc[i] - zlema[i - 1])

        # Convert the ZLEMA to a Pandas Series.
        data['ZLEMA'] = zlema

        # zlema_signal = np.zeros(shape=(len(zlema),))

        # for i in range(len(zlema)):
        #   if data['Close'].iloc[i] > data['ZLEMA'].iloc[i]:
        #     zlema_signal[i] = 1
        #   elif data['Close'].iloc[i] < data['ZLEMA'].iloc[i]:
        #     zlema_signal[i] = -1
        #   else:
        #     zlema_signal[i] = 0

        # data['ZLEMA_Signal'] = zlema_signal

        # Define tolerance as a percentage of the close price
        tolerance = 0.02 # for example, 1%

        # # Initialize column with 0s
        # data['ZLEMA_Signal'] = 0

        # # Buy signal when Close is above (1 + tolerance) * ZLEMA
        # data.loc[data['Close'] > data['ZLEMA'] * (1 + tolerance), 'ZLEMA_Signal'] = 1

        # # Sell signal when Close is below (1 - tolerance) * ZLEMA
        # data.loc[data['Close'] < data['ZLEMA'] * (1 - tolerance), 'ZLEMA_Signal'] = -1

        # Calculate the first exponential moving average (EMA1).
        ema1 = data['Close'].ewm(span=period, min_periods=period).mean()

        # Calculate the second exponential moving average (EMA2).
        ema2 = ema1.ewm(span=period, min_periods=period).mean()

        # Calculate the third exponential moving average (EMA3).
        ema3 = ema2.ewm(span=period, min_periods=period).mean()

        # Calculate the TEMA.
        tema = 3 * ema1 - 3 * ema2 + ema3

        # Convert the TEMA to a Pandas Series.
        data['TEMA'] = tema

        # tema_signal = np.zeros(shape=(len(tema),))

        # for i in range(len(tema)):
        #   if data['Close'].iloc[i] > data['TEMA'].iloc[i]:
        #     tema_signal[i] = 1
        #   elif data['Close'].iloc[i] < data['TEMA'].iloc[i]:
        #     tema_signal[i] = -1
        #   else:
        #     tema_signal[i] = 0

        # data['TEMA_Signal'] = tema_signal

        # # Initialize column with 0s
        # data['TEMA_Signal'] = 0


        # # Buy signal when Close is above (1 + tolerance) * TEMA
        # data.loc[data ['Close']> data ['TEMA']* (1 + tolerance), 'TEMA_Signal']= 1

        # # Sell signal when Close is below (1 - tolerance) * TEMA
        # data.loc[data ['Close']< data ['TEMA']*(1-tolerance), 'TEMA_Signal']= -1

        # Calculate RSI and its signal
        delta = data['Close'].diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0

        average_gain = up.rolling(window=rsi_period).mean()
        average_loss = abs(down.rolling(window=rsi_period).mean())

        rs = average_gain / average_loss

        data['RSI'] =100 - (100 / (1 + rs))
        data["RSI_Signal"] =0
        data.loc[data.RSI < 30,"RSI_Signal"] =1
        data.loc[data.RSI > 70,"RSI_Signal"] =-1

        # Calculate MACD Line: (12-day EMA - 26-day EMA)
        EMA_short = data['Close'].ewm(span=short_ema_period).mean()
        EMA_long = data['Close'].ewm(span=long_ema_period).mean()
        data['MACD_Line'] = EMA_short - EMA_long

        # Calculate Signal Line: a n-day MA of MACD Line
        data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

        # Generate MACD signals based on crossovers
        data["MACD_Signal"] = 0 # Neutral
        data.loc[(data.MACD_Line > data.Signal_Line) & (data.MACD_Line.shift() < data.Signal_Line.shift()),"MACD_Signal"] = 1 # Buy
        data.loc[(data.MACD_Line < data.Signal_Line) & (data.MACD_Line.shift() > data.Signal_Line.shift()),"MACD_Signal"] = -1 # Sell

        #Calculate Exponential Moving Average(EMA) and generate signals based on crossovers
        short_EMA =(data.Close.ewm(span=short_ema_period, adjust=False).mean())
        long_EMA =(data.Close.ewm(span=long_ema_period, adjust=False).mean())

        # Generate EMA signals based on crossovers
        data["EMA_Short"]=short_EMA;
        data["EMA_Long"]=long_EMA;
        # Default to Neutral, then if short EMA is greater than long EMA, signal to Buy. If short EMA is less than long EMA, signal to Sell.

        # Define a small percentage as a threshold
        ema_diff_threshold = 0.01 # 1% difference

        # Calculate absolute percent difference between short and long EMA
        data['EMA_Diff'] = abs((data['EMA_Short'] - data['EMA_Long']) / data['EMA_Long'])

        # Default all to neutral signal (0)
        data["EMA_Signal"] = 0

        # If absolute percent difference is greater than threshold, assign buy or sell signal
        data.loc[(data['EMA_Short'] > data['EMA_Long']) & (data['EMA_Diff'] > ema_diff_threshold), "EMA_Signal"] = 1   # "Buy"
        data.loc[(data['EMA_Short'] < data['EMA_Long']) & (data['EMA_Diff'] > ema_diff_threshold), "EMA_Signal"] = -1  # "Sell"


        # Calculate Volatility as rolling standard deviation of log returns
        data["Log_Return"] =(np.log(data.Close).diff())
        data["Volatility"] =(data.Log_Return.rolling(window=vol_period, center=False).std())

        # # Generate Volatility signals based on high or low volatility
        # high_vol_threshold = 0.025
        # low_vol_threshold = 0.015
        # data['Volatility_Signal'] = 0
        # data.loc[data.Volatility > high_vol_threshold, 'Volatility_Signal'] = 1
        # data.loc[data.Volatility < low_vol_threshold, 'Volatility_Signal'] = -1

        #Calculate Bollinger Bands
        middle_band =data.Close.rolling(window=bollinger_band_window_size, center=False).mean()
        std_dev =data.Close.rolling(window=bollinger_band_window_size, center=False).std()
        data["Upper_Band"] =(middle_band + std_dev*2)
        data["Lower_Band"] =(middle_band - std_dev*2)

        # Generate Bollinger Band signals based on crossovers
        data['BBand_Signal'] = 0
        data.loc[(data.Close > data.Upper_Band), "BBand_Signal"]=-1
        data.loc[(data.Close < data.Lower_Band), "BBand_Signal"]=1

        # Drop unnecessary columns
        data = data.drop(columns=['Open', 'Low', 'High'])

    else:
        print("'Close' column is not present in the input dataframe.")

    return data

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col

    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)

    data[data.columns] = scaler().fit_transform(data)

    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    # return data.drop(['Adj Close', 'Volume'], axis=1)
    return data.drop('Adj Close', axis=1)

def prepare_data_separate(train_data_list, valid_data_list, X_seq_len, y_seq_len, target_col, symbol, start_date, end_date,
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        # train_data = train_data.copy()

        # # Create separate dataframes for prices and volume
        # train_data_reshaped = train_data.values.reshape(-1, 1)

        # train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # # Reshape it back to original shape.
        # train_data[train_data.columns] = train_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # train_data['Target'] = train_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # train_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # train_data.drop(target_col_name, axis=1, inplace=True)


        train_data = train_data.copy()
        train_data = calculate_indicators(train_data)

        train_data['Target'] = train_data[target_col_name].shift(forward)
        train_data.dropna(inplace=True)
        train_data = train_data.drop(target_col_name, axis=1)

        train_data[train_data.columns] = scaler.fit_transform(train_data)


        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:

        # valid_data = valid_data.copy()

        # # Create separate dataframes for prices and volume
        # valid_data_reshaped = valid_data.values.reshape(-1, 1)

        # valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # # Reshape it back to original shape.
        # valid_data[valid_data.columns] = valid_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # valid_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # valid_data.drop(target_col_name, axis=1, inplace=True)

        valid_data = valid_data.copy()
        valid_data = calculate_indicators(valid_data)

        valid_data['Target'] = valid_data[target_col_name].shift(forward)
        valid_data.dropna(inplace=True)
        valid_data = valid_data.drop(target_col_name, axis=1)
        valid_data[valid_data.columns] = scaler.fit_transform(valid_data)

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)

    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)

    # test_data_unshifted = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_unshifted_reshaped = test_data_unshifted.values.reshape(-1, 1)

    # test_data_unshifted_transformed = scaler.fit_transform(test_data_unshifted_reshaped)

    # # Reshape it back to original shape.
    # test_data_unshifted[test_data_unshifted.columns] = test_data_unshifted_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]

    # # Drop NA values if there are any due to shifting.
    # test_data_unshifted.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_unshifted.drop(target_col_name, axis=1, inplace=True)

    # test_data = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_reshaped = test_data.values.reshape(-1, 1)

    # test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # # Reshape it back to original shape.
    # test_data[test_data.columns] = test_data_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data['Target'] = test_data[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data.drop(target_col_name, axis=1, inplace=True)

    # # Fetch a fresh copy of the test data
    # test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)

    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)

    # test_data_unshifted = test_data.copy()
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]
    # test_data_unshifted.dropna(inplace=True)
    # test_data_unshifted = test_data_unshifted.drop(target_col_name, axis=1)
    # test_data_unshifted[test_data_unshifted.columns] = scaler.fit_transform(test_data_unshifted)

    # Scale test data
    test_data = test_data.copy()
    test_data = calculate_indicators(test_data)

    test_data[test_data.columns] = scaler.fit_transform(test_data)
    test_data['Target'] = test_data[target_col_name].shift(forward)
    test_data.dropna(inplace=True)
    test_data = test_data.drop(target_col_name, axis=1)

    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)

    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized = calculate_indicators(test_data_short_unnormalized)

    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)

    # test_data_short = test_data_short.copy()

    # # Create separate dataframes for prices and volume
    # test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    # test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # # Reshape it back to original shape.
    # test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data_short.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_short.drop(target_col_name, axis=1, inplace=True)

    # Scale test data
    test_data_short = test_data_short.copy()
    test_data_short = calculate_indicators(test_data_short)
    test_data_short[test_data_short.columns] = scaler.fit_transform(test_data_short)
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)
    test_data_short.dropna(inplace=True)
    test_data_short = test_data_short.drop(target_col_name, axis=1)

    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, X_seq_len, y_seq_len

def prepare_data_common(train_data, valid_data, test_data, test_data_short, X_seq_len, y_seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, X_seq_len, y_seq_len)
    X_valid, y_valid = create_sequences(valid_data, X_seq_len, y_seq_len)
    X_test, y_test = create_sequences(test_data, X_seq_len, y_seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, X_seq_len, y_seq_len)


    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short

# class DecoderOnlyTransformerModel(nn.Module):
#     def __init__(self, n_features, nhead=8, nhid=64, nlayers=6, dropout=0.1,
#                  l1_regularization=0, l2_regularization=0, activation_function=torch.nn.ReLU(), y_seq_len=5):
#         super(DecoderOnlyTransformerModel, self).__init__()

#         self.pos_encoder = nn.Sequential(
#             nn.Linear(n_features, nhid),
#             activation_function,
#             nn.Linear(nhid, nhid),
#             activation_function
#         )

#         decoder_layers = TransformerDecoderLayer(nhid, nhead)
#         self.transformer_decoder = TransformerDecoder(decoder_layers,nlayers)

#         self.decoder = nn.Linear(nhid,n_features)

#         self.l1_regularization = l1_regularization
#         self.l2_regularization = l2_regularization
#         self.y_seq_len = y_seq_len

#     def init_weights(self):
#         # initrange = 0.1
#         nn.init.xavier_uniform_(self.pos_encoder[0].weight)
#         nn.init.xavier_uniform_(self.pos_encoder[2].weight)
#         self.decoder.bias.data.zero_()
#         nn.init.xavier_uniform_(self.decoder.weight)

#     def regularization_loss(self):
#         l1_loss = 0
#         l2_loss = 0
#         for param in self.parameters():
#             l1_loss += torch.norm(param, 1)
#             l2_loss += torch.norm(param, 2) ** 2
#         return self.l1_regularization * l1_loss + self.l2_regularization * l2_loss

#     def generate_square_subsequent_mask(self, sz):
#         mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
#         mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#         return mask

#     def forward(self, x):
#         x = self.pos_encoder(x)
#         output = torch.zeros_like(x)

#         # Process in chunks
#         chunk_size = 100  # Set based on your available memory
#         for i in range(0, x.size(0), chunk_size):
#             x_chunk = x[i:i+chunk_size]
#             tgt_mask = self.generate_square_subsequent_mask(x_chunk.size(0)).to(x.device)
#             output_chunk = self.transformer_decoder(x_chunk, x_chunk, tgt_mask=tgt_mask)
#             output[i:i+chunk_size] = output_chunk

#         output = self.decoder(output)

#         return output[:,-self.y_seq_len:,:]

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2, y_seq_len=5):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        self.y_seq_len = y_seq_len
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-self.y_seq_len:,:])
        # output = output.unsqueeze(1)
        
        return output

def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)
            reg_loss = model.regularization_loss()
            total_loss = loss + reg_loss
            #print(loss.device)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(total_loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

# def evaluate_model(model, X, y, use_target_col=True):
#     torch.cuda.empty_cache()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)

#     X = X.to(device)
#     y = y.to(device)

#     with torch.no_grad():
#         y_pred = model(X)

#         # Reshape the tensors to 2D and move them back to the CPU before computing metrics
#         y = y.view(-1, y.shape[-1]).cpu()
#         y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

#         if use_target_col:
#             y = y[:,-1] # Pick the last column (target column)
#             y_pred = y_pred[:,-1]

#         mse = mean_squared_error(y, y_pred)
#         mae = mean_absolute_error(y, y_pred)
#         r2 = r2_score(y, y_pred)

#     return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()

def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def evaluate_and_plot_predictions(model, X_test, y_test, trial, y_seq_len=1, use_target_col=True, use_target_col_plot=False, metrics_use=True,
                                  plots_use=True,save_directory=None, future_predictions=None, scaler=None, col_names=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]

    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)

    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()

    # Reshape tensors for metrics computation
    y_pred_for_metrics = output.view(-1, output.shape[-1])
    y_for_metrics = y_test.view(-1, y_test.shape[-1])

    if metrics_use:
        
        evaluation_results = {}

        # Iterate over each column in y_true
        for i, col_name in enumerate(col_names):
            # Calculate metrics for this column
            mse = mean_squared_error(y_for_metrics[:,i], y_pred_for_metrics[:,i])
            mae = mean_absolute_error(y_for_metrics[:,i], y_pred_for_metrics[:,i])
            r2 = r2_score(y_for_metrics[:,i], y_pred_for_metrics[:,i])

            # Store results in dictionary
            evaluation_results[col_name] = {
                'mse': mse,
                'mae': mae,
                'r2': r2,
            }

    if plots_use:
        y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
        output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)

        # If use_target_col is True, only plot the target column, otherwise plot all feature columns
        if use_target_col_plot:
            last_future_prediction = future_predictions[-y_seq_len:]
            print("future_predictions:", last_future_prediction[:, -1])

            last_future_prediction_expanded = np.expand_dims(last_future_prediction[:, -1], axis=1)
            # Transpose last_future_prediction_expanded
            last_future_prediction_transposed = np.transpose(last_future_prediction_expanded)
            # Now concatenate
            combined_predictions = np.concatenate([output_org[:, -y_seq_len:, -1], last_future_prediction_transposed])

            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(y_test_org[:, -y_seq_len:, -1])), y_test_org[:, -y_seq_len:, -1], label='Actual')
            plt.plot(np.arange(len(combined_predictions)), combined_predictions, label='Predicted + Future Predicted')

            plt.xlabel('Time Step')
            plt.ylabel('Value')
            # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_names[-1]} (Trial {trial+1}_{test_length})')
            plt.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
                plt.savefig(save_path)
            plt.show()
        else:
            for j in range(n_features):
                last_future_prediction = future_predictions[-y_seq_len:]
                print("future_predictions:", last_future_prediction[:, j])
                # Add an extra dimension to last_future_prediction
                last_future_prediction_expanded = np.expand_dims(last_future_prediction[:, j], axis=1)
                # Transpose last_future_prediction_expanded
                last_future_prediction_transposed = np.transpose(last_future_prediction_expanded)

                # Now concatenate
                combined_predictions = np.concatenate([output_org[:, -y_seq_len:, j], last_future_prediction_transposed])

                fig, ax = plt.subplots(figsize=(15, 8))
                ax.plot(np.arange(len(y_test_org[:, -y_seq_len:, j])), y_test_org[:, -y_seq_len:, j], label='Actual')
                ax.plot(np.arange(len(combined_predictions)), combined_predictions, label='Predicted + Future Predicted')

                ax.set_xlabel('Time Step')
                ax.set_ylabel('Value')
                # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
                ax.set_title(f'Actual and Predicted Values for {col_names[j]} (Trial {trial+1}_{test_length})')
                ax.legend()

                if save_directory:
                    save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                    plt.savefig(save_path)
                plt.show()

    return evaluation_results

def calculate_metrics_all(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)

    return mse, mae, r2

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray, col_names=None):
    # Ensure y_true and y_pred are numpy arrays
    assert isinstance(y_true, np.ndarray), "y_true must be a numpy array"
    assert isinstance(y_pred, np.ndarray), "y_pred must be a numpy array"

    col_names = col_names.tolist()

    # Ensure col_names is a list and has correct length
    assert isinstance(col_names, list), "col_names must be a list"
    assert len(col_names) == y_true.shape[1], "col_names must have same length as number of columns in y_true"

    # Initialize an empty dictionary to store results
    evaluation_results = {}

    # Iterate over each column in y_true
    for i, col_name in enumerate(col_names):
        # Calculate metrics for this column
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        r2 = r2_score(y_true[:, i], y_pred[:, i])

        # Store results in dictionary
        evaluation_results[col_name] = {
            'mse': mse,
            'mae': mae,
            'r2': r2,
        }

    return evaluation_results

def predict_future(model, X_test, y_test, n_last_sequence=1, scaler=None, y_seq_len=5):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
    #     return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)

    def new_sequence(last_sequences, y_test_sequences, sequence_length):
        return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test_sequences[:, :, :]], axis=1)

    # Prepare the most recent input sequence
    x_test_sequences = X_test[-(n_last_sequence):, :, :]
    y_test_sequences = y_test[-(n_last_sequence):, -1, :]
    y_test_sequences = y_test_sequences.reshape(y_test_sequences.shape[0], 1, y_test_sequences.shape[1])

    last_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = torch.Tensor(last_sequences)

    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
      # Generate a prediction
        recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features)
        with torch.no_grad():
          input_seq = torch.Tensor(recent_input_sequence).to(device)
          output = model(input_seq).cpu().numpy()

          future_predictions = output[0, :, :]

        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

    return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True,
                  overall_future_plot=True, future_predictions=None,
                  use_target_col=True, use_target_col_plot=False, metrics_use=True, plots_use=True,
                  train_data_list=None, valid_data_list=None, symbol=None, start_date=None, end_date=None,
                  start_date_short=None, end_date_short=None, valid_size=0.5, X_seq_len=10, y_seq_len=5, n_last_sequence=1, forward=-1):

    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    short_all_future_predictions = []
    all_future_predictions = []
    all_results_metrics = []
    all_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")

        start = timeit.default_timer()

        # Generate random hyperparameters and parameters
        X_seq_len = random.choice(range(10, 21))
        y_seq_len = random.choice(range(3, 4))
        nlayers = random.choice(range(1, 4))
        nneurons = random.choice(range(200, 301))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(500, 1001))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001])
        patience = random.choice(range(5, 6))
        min_delta = random.choice([0.0001])
        l2_regularization = random.choice([0])
        # n_predict = random.choice(range(5, 6))
        # n_last_sequence = random.choice(range(100, 101))
        # forward = -n_predict

        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, X_seq_len=X_seq_len, y_seq_len=y_seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, X_seq_len, y_seq_len = prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short,
                                                                                        end_date_short=end_date_short, X_seq_len=X_seq_len, y_seq_len=y_seq_len, target_col=target_col)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data,
                                                                                                                test_data_short=test_data_short, X_seq_len=X_seq_len, y_seq_len=y_seq_len)

        input_shape = (X_train.shape[0], X_seq_len, X_train.shape[2])        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout, y_seq_len=y_seq_len)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer,
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)

        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()


        # # Inverse transform the y_test to the original scale
        # test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)
        # test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)

        # test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        # test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)

        # Inverse transform the y_test to the original scale
        test_scaler = StandardScaler().fit(test_data_unnormalized)

        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized)

        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_last_sequence > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test, y_test, n_last_sequence=n_last_sequence, scaler=test_scaler)
        future_predictions = np.squeeze(future_predictions)
        future_predictions_org = np.squeeze(future_predictions_org)
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(y_seq_len):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Generate future predictions
        if n_last_sequence > 0:
            short_future_predictions, short_future_predictions_org = predict_future(loaded_model, X_test_short, y_test_short, n_last_sequence=n_last_sequence, scaler=test_scaler_short)
        short_future_predictions = np.squeeze(future_predictions)
        short_future_predictions_org = np.squeeze(future_predictions_org)
        short_future_predictions_df = pd.DataFrame(short_future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test_short.shape[2])])
        short_future_predictions_all_features = short_future_predictions_df.iloc[-(y_seq_len):]
        short_future_predictions_target = short_future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        short_future_predictions_target_df = short_future_predictions_target.to_frame(name='Future_Predicted_Target')
        short_future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        short_all_future_predictions.append(short_future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        short_all_future_predictions_df = pd.concat(short_all_future_predictions, axis=0)
        print(f"Future Predictions_Short (Trial {trial+1}): {short_future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:


            train_metrics = evaluate_and_plot_predictions(loaded_model, X_train, y_train, trial, y_seq_len=y_seq_len, use_target_col=use_target_col,
                            use_target_col_plot=use_target_col_plot, metrics_use=True, plots_use=False, save_directory=save_directory,
                            scaler=test_scaler, col_names=col_label, test_length="long period",
                            future_predictions=future_predictions if len(future_predictions) > 0 else None)

            test_metrics = evaluate_and_plot_predictions(loaded_model, X_test, y_test, trial, y_seq_len=y_seq_len, use_target_col=use_target_col,
                            use_target_col_plot=use_target_col_plot, metrics_use=True, plots_use=True, save_directory=save_directory,
                            scaler=test_scaler, col_names=col_label, test_length="long period",
                            future_predictions=future_predictions if len(future_predictions) > 0 else None)

            test_metrics_short = evaluate_and_plot_predictions(loaded_model, X_test_short, y_test_short, trial, y_seq_len=y_seq_len, use_target_col=use_target_col,
                            use_target_col_plot=use_target_col_plot, metrics_use=True, plots_use=True, save_directory=save_directory,
                            scaler=test_scaler_short, col_names=col_label, test_length="short period",
                            future_predictions=short_future_predictions if len(short_future_predictions) > 0 else None)

        # Add the results to the results dataframe
        params = {"X_seq_len": X_seq_len, "y_seq_len": y_seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_last_sequence": n_last_sequence, "forward": forward}
        
        results_metrics = {**params, "trial": trial+1}

        # Add metrics for each feature

        for col_name, metrics in train_metrics.items():
            results_metrics[f"{col_name} Train MSE"] = np.round(metrics['mse'], 5)
            results_metrics[f"{col_name} Train MAE"] = np.round(metrics['mae'], 5)
            results_metrics[f"{col_name} Train R2"] = np.round(metrics['r2'], 5)

        for col_name, metrics in test_metrics.items():
            results_metrics[f"{col_name} Test MSE"] = np.round(metrics['mse'], 5)
            results_metrics[f"{col_name} Test MAE"] = np.round(metrics['mae'], 5)
            results_metrics[f"{col_name} Test R2"] = np.round(metrics['r2'], 5)

        for col_name, metrics in test_metrics_short.items():
            results_metrics[f"{col_name} Test Short MSE"] = np.round(metrics['mse'], 5)
            results_metrics[f"{col_name} Test Short MAE"] = np.round(metrics['mae'], 5)
            results_metrics[f"{col_name} Test Short R2"] = np.round(metrics['r2'], 5)

        all_results_metrics.append(results_metrics)

        all_results_metrics_df = pd.concat([pd.DataFrame(data=d, index=[0]) for d in all_results_metrics], axis=0)

        print(all_results_metrics_df)

        if save_directory:
            all_results_metrics_df.to_csv(os.path.join(save_directory, f"all_results_{trial}.csv"))

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        # Initialize accumulators
        accumulated_y_true_org = []
        accumulated_y_pred_org = []
        accumulated_y_true = []
        accumulated_y_pred = []

        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > y_seq_len:

                    y_true_org = y_test_org[-(n_last_sequence-i):-((n_last_sequence-i)-y_seq_len), -1]
                    y_pred_org = np.array(future_predictions[-(y_seq_len*(n_last_sequence-i)):-(y_seq_len*(n_last_sequence-(i+1)))])

                    # Inverse transform the y_test to the original scale
                    y_true = y_test[-(n_last_sequence-i):-((n_last_sequence-i)-y_seq_len), -1]
                    y_pred = np.array(future_predictions_org[-(y_seq_len*(n_last_sequence-i)):-(y_seq_len*(n_last_sequence-(i+1)))])

                else:
                    y_true_org = y_test_org[-(n_last_sequence-i):, -1]
                    y_pred_org = np.array(future_predictions[-(y_seq_len*(n_last_sequence-i)):-((y_seq_len*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(y_seq_len)))])

                    # Inverse transform the y_test to the original scale
                    y_true = y_test[-(n_last_sequence-i):, -1]
                    y_pred = np.array(future_predictions_org[-(y_seq_len*(n_last_sequence-i)):-((y_seq_len*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(y_seq_len)))])

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_org.append(y_true_org)
                accumulated_y_pred_org.append(y_pred_org)
                accumulated_y_true.append(y_true)
                accumulated_y_pred.append(y_pred)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_org = np.concatenate(accumulated_y_true_org)
        accumulated_y_pred_org = np.concatenate(accumulated_y_pred_org)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred = np.concatenate(accumulated_y_pred)

        # Calculate overall metrics
        predict_metrics_org = calculate_metrics(accumulated_y_true_org, accumulated_y_pred_org, col_names=col_label)
        predict_metrics = calculate_metrics(accumulated_y_true, accumulated_y_pred, col_names=col_label)

        mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics_all(accumulated_y_true_org, accumulated_y_pred_org)
        mse_all_features, mae_all_features, r2_all_features = calculate_metrics_all(accumulated_y_true ,accumulated_y_pred)

        # error_percentage = (mae/accumulated_y_true.mean())*100
        error_percentage_all_features = (mae_all_features/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        future_metrics  ={
            "Trial": [trial],
            # "Future MSE (org)": [np.round(mse_org, 5)],
            # "Future MAE (org)": [np.round(mae_org, 5)],
            # "Future R2 (org)": [np.round(r2_org, 5)],
            "Future MSE (org all features)": [np.round(mse_org_all_features , 5)],
            "Future MAE (org all features)": [np.round(mae_org_all_features, 5)],
            "Future R2 (org all features)": [np.round(r2_org_all_features , 5)],
            # "Future MSE": [np.round(mse, 5)],
            # "Future MAE": [np.round(mae, 5)],
            # "Future R2": [np.round(r2, 5)],
            "Future MSE (all features)": [np.round(mse_all_features, 5)],
            "Future MAE (all features)": [np.round(mae_all_features, 5)],
            "Future R2 (all features)": [np.round(r2_all_features, 5)],
            # "Future Error Percentage": [np.round(error_percentage, 3)],
            "Future Error Percentage (all features)": [np.round(error_percentage_all_features, 3)]
        }
        # Add metrics for each feature
        for col_name, metrics in predict_metrics_org.items():
            future_metrics[f"{col_name} MSE (org)"] = np.round(metrics['mse'], 5)
            future_metrics[f"{col_name} MAE (org)"] = np.round(metrics['mae'], 5)
            future_metrics[f"{col_name} R2 (org)"] = np.round(metrics['r2'], 5)

        for col_name, metrics in predict_metrics.items():
            future_metrics[f"{col_name} MSE"] = np.round(metrics['mse'], 5)
            future_metrics[f"{col_name} MAE"] = np.round(metrics['mae'], 5)
            future_metrics[f"{col_name} R2"] = np.round(metrics['r2'], 5)

        all_future_metrics.append(future_metrics)

        all_future_metrics_df = pd.concat([pd.DataFrame(data=d, index=[0]) for d in all_future_metrics], axis=0)

        print(all_future_metrics_df)

        if save_directory:
            all_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:
            if use_target_col:
                combined_predictions = np.concatenate((accumulated_y_pred_org[:,-1], future_predictions_target))
                plt.figure(figsize=(15,8))
                plt.plot(np.arange(len(accumulated_y_true_org[:,-1])),
                        accumulated_y_true_org[:,-1], label='Actual')
                plt.plot(np.arange(len(combined_predictions)),
                        combined_predictions, label='Predicted')
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                plt.title(f'Overall Actual and Predicted Values for {col_label[-1]} (Trial {trial+1})')
                plt.legend()

                # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
                if save_directory:
                    save_path=os.path.join(save_directory,
                                        f"overall_predictions_plot_Trial{trial+1}.png")
                    plt.savefig(save_path)
                plt.show()
            else:
                for j in range(X_test.shape[2]):
                    combined_predictions = np.concatenate((accumulated_y_pred_org[:, j], future_predictions_all_features.iloc[:, j]))
                    fig, ax = plt.subplots(figsize=(15, 8))
                    ax.plot(np.arange(len(accumulated_y_true_org[:, j])),
                        accumulated_y_true_org[:, j], label='Actual')
                    ax.plot(np.arange(len(combined_predictions)),
                        combined_predictions, label='Predicted + Future Predicted')

                    ax.set_xlabel('Time Step')
                    ax.set_ylabel('Value')
                    # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
                    ax.set_title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1})')
                    ax.legend()

                    if save_directory:
                        save_path = os.path.join(save_directory, f"predictions_plot_var_{j+1}_Trial{trial+1}.png")
                        plt.savefig(save_path)
                    plt.show()

        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")

    return all_results_metrics_df, all_future_predictions_df, all_future_metrics_df

In [ ]:
# Chip Maker

import yfinance as yf

start_date = '2018-01-01'
end_date = '2023-09-26'
symbol = 'TSM'
TSM = yf.download(symbol, start=start_date, end=end_date)
TSM = TSM.drop('Adj Close', axis=1)
symbol = 'INTC'
INTC = yf.download(symbol, start=start_date, end=end_date)
INTC = INTC.drop('Adj Close', axis=1)
symbol = 'ASML'
ASML = yf.download(symbol, start=start_date, end=end_date)
ASML = ASML.drop('Adj Close', axis=1)
symbol = 'MU'
MU = yf.download(symbol, start=start_date, end=end_date)
MU = MU.drop('Adj Close', axis=1)
symbol = 'NVDA'
NVDA = yf.download(symbol, start=start_date, end=end_date)
NVDA = NVDA.drop('Adj Close', axis=1)
symbol = 'AMD'
AMD = yf.download(symbol, start=start_date, end=end_date)
AMD = AMD.drop('Adj Close', axis=1)
symbol = 'QCOM'
QCOM = yf.download(symbol, start=start_date, end=end_date)
QCOM = QCOM.drop('Adj Close', axis=1)
symbol = 'SNPS'
SNPS = yf.download(symbol, start=start_date, end=end_date)
SNPS = SNPS.drop('Adj Close', axis=1)
symbol = 'MRVL'
MRVL = yf.download(symbol, start=start_date, end=end_date)
MRVL = MRVL.drop('Adj Close', axis=1)
symbol = '^IXIC'
IXIC = yf.download(symbol, start=start_date, end=end_date)
IXIC = IXIC.drop('Adj Close', axis=1)

# information technology

symbol = 'AAPL'
AAPL = yf.download(symbol, start=start_date, end=end_date)
AAPL = AAPL.drop('Adj Close', axis=1)
symbol = 'MSFT'
MSFT = yf.download(symbol, start=start_date, end=end_date)
MSFT = MSFT.drop('Adj Close', axis=1)
symbol = 'TSLA'
TSLA = yf.download(symbol, start=start_date, end=end_date)
TSLA = TSLA.drop('Adj Close', axis=1)
symbol = 'GOOGL'
GOOGL = yf.download(symbol, start=start_date, end=end_date)
GOOGL = GOOGL.drop('Adj Close', axis=1)
symbol = 'GOOG'
GOOG = yf.download(symbol, start=start_date, end=end_date)
GOOG = GOOG.drop('Adj Close', axis=1)
symbol = 'AMZN'
AMZN = yf.download(symbol, start=start_date, end=end_date)
AMZN = AMZN.drop('Adj Close', axis=1)
symbol = 'META'
META = yf.download(symbol, start=start_date, end=end_date)
META = META.drop('Adj Close', axis=1)
symbol = 'AMD'
AMD = yf.download(symbol, start=start_date, end=end_date)
AMD = AMD.drop('Adj Close', axis=1)
symbol = 'ASML'
ASML = yf.download(symbol, start=start_date, end=end_date)
ASML = ASML.drop('Adj Close', axis=1)
symbol = 'NVDA'
NVDA = yf.download(symbol, start=start_date, end=end_date)
NVDA = NVDA.drop('Adj Close', axis=1)
symbol = 'IBM'
IBM = yf.download(symbol, start=start_date, end=end_date)
IBM = IBM.drop('Adj Close', axis=1)
symbol = 'NFLX'
NFLX = yf.download(symbol, start=start_date, end=end_date)
NFLX = NFLX.drop('Adj Close', axis=1)

# Consumer


symbol = 'WMT'
WMT = yf.download(symbol, start=start_date, end=end_date)
WMT = WMT.drop('Adj Close', axis=1)
symbol = 'TGT'
TGT = yf.download(symbol, start=start_date, end=end_date)
TGT = TGT.drop('Adj Close', axis=1)
symbol = 'COST'
COST = yf.download(symbol, start=start_date, end=end_date)
COST = COST.drop('Adj Close', axis=1)
symbol = 'HD'
HD = yf.download(symbol, start=start_date, end=end_date)
HD = HD.drop('Adj Close', axis=1)
symbol = 'LOW'
LOW = yf.download(symbol, start=start_date, end=end_date)
LOW = LOW.drop('Adj Close', axis=1)

symbol = 'PG'
PG = yf.download(symbol, start=start_date, end=end_date)
PG = PG.drop('Adj Close', axis=1)
symbol = 'JNJ'
JNJ = yf.download(symbol, start=start_date, end=end_date)
JNJ = JNJ.drop('Adj Close', axis=1)
symbol = 'PFE'
PFE = yf.download(symbol, start=start_date, end=end_date)
PFE = PFE.drop('Adj Close', axis=1)
symbol = 'CVS'
CVS = yf.download(symbol, start=start_date, end=end_date)
CVS = CVS.drop('Adj Close', axis=1)

symbol = 'KO'
KO = yf.download(symbol, start=start_date, end=end_date)
KO = KO.drop('Adj Close', axis=1)
symbol = 'PEP'
PEP = yf.download(symbol, start=start_date, end=end_date)
PEP = PEP.drop('Adj Close', axis=1)

symbol = 'NKE'
NKE = yf.download(symbol, start=start_date, end=end_date)
NKE = NKE.drop('Adj Close', axis=1)
symbol = 'MCD'
MCD = yf.download(symbol, start=start_date, end=end_date)
MCD = MCD.drop('Adj Close', axis=1)
symbol = 'SBUX'
SBUX = yf.download(symbol, start=start_date, end=end_date)
SBUX = SBUX.drop('Adj Close', axis=1)

symbol = 'VZ'
VZ = yf.download(symbol, start=start_date, end=end_date)
VZ = VZ.drop('Adj Close', axis=1)
symbol = 'T'
T = yf.download(symbol, start=start_date, end=end_date)
T = T.drop('Adj Close', axis=1)
symbol = 'FOX'
FOX = yf.download(symbol, start=start_date, end=end_date)
FOX = FOX.drop('Adj Close', axis=1)
symbol = 'WBD'
WBD = yf.download(symbol, start=start_date, end=end_date)
WBD = WBD.drop('Adj Close', axis=1)
symbol = 'DIS'
DIS = yf.download(symbol, start=start_date, end=end_date)
DIS = DIS.drop('Adj Close', axis=1)

symbol = 'UPS'
UPS = yf.download(symbol, start=start_date, end=end_date)
UPS = UPS.drop('Adj Close', axis=1)
symbol = 'FDX'
FDX = yf.download(symbol, start=start_date, end=end_date)
FDX = FDX.drop('Adj Close', axis=1)
symbol = 'DAL'
DAL = yf.download(symbol, start=start_date, end=end_date)
DAL = DAL.drop('Adj Close', axis=1)
symbol = 'AAL'
AAL = yf.download(symbol, start=start_date, end=end_date)
AAL = AAL.drop('Adj Close', axis=1)
symbol = 'XOM'
XOM = yf.download(symbol, start=start_date, end=end_date)
XOM = XOM.drop('Adj Close', axis=1)
symbol = 'CVX'
CVX = yf.download(symbol, start=start_date, end=end_date)
CVX = CVX.drop('Adj Close', axis=1)

symbol = 'BAC'
BAC = yf.download(symbol, start=start_date, end=end_date)
BAC = BAC.drop('Adj Close', axis=1)
symbol = 'JPM'
JPM = yf.download(symbol, start=start_date, end=end_date)
JPM = JPM.drop('Adj Close', axis=1)
symbol = 'MA'
MA = yf.download(symbol, start=start_date, end=end_date)
MA = MA.drop('Adj Close', axis=1)
symbol = 'V'
V = yf.download(symbol, start=start_date, end=end_date)
V = V.drop('Adj Close', axis=1)
symbol = 'SPG'
SPG = yf.download(symbol, start=start_date, end=end_date)
SPG = SPG.drop('Adj Close', axis=1)
symbol = 'VNO'
VNO = yf.download(symbol, start=start_date, end=end_date)
VNO = VNO.drop('Adj Close', axis=1)

symbol = 'MMM'
MMM = yf.download(symbol, start=start_date, end=end_date)
MMM = MMM.drop('Adj Close', axis=1)
symbol = 'GE'
GE = yf.download(symbol, start=start_date, end=end_date)
GE = GE.drop('Adj Close', axis=1)
symbol = 'F'
F = yf.download(symbol, start=start_date, end=end_date)
F = F.drop('Adj Close', axis=1)
symbol = 'GM'
GM = yf.download(symbol, start=start_date, end=end_date)
GM = GM.drop('Adj Close', axis=1)
symbol = 'HON'
HON = yf.download(symbol, start=start_date, end=end_date)
HON = HON.drop('Adj Close', axis=1)
symbol = 'LMT'
LMT = yf.download(symbol, start=start_date, end=end_date)
LMT = LMT.drop('Adj Close', axis=1)


# Futures



symbol = 'ES=F'
ESF = yf.download(symbol, start=start_date, end=end_date)
ESF = ESF.drop('Adj Close', axis=1)
symbol = 'YM=F'
YMF = yf.download(symbol, start=start_date, end=end_date)
YMF = YMF.drop('Adj Close', axis=1)
symbol = 'NQ=F'
NQF = yf.download(symbol, start=start_date, end=end_date)
NQF = NQF.drop('Adj Close', axis=1)
symbol = 'RTY=F'
RTYF = yf.download(symbol, start=start_date, end=end_date)
RTYF = RTYF.drop('Adj Close', axis=1)
symbol = 'ZB=F'
ZBF = yf.download(symbol, start=start_date, end=end_date)
ZBF = ZBF.drop('Adj Close', axis=1)
symbol = 'ZN=F'
ZNF = yf.download(symbol, start=start_date, end=end_date)
ZNF = ZNF.drop('Adj Close', axis=1)
symbol = 'ZF=F'
ZFF = yf.download(symbol, start=start_date, end=end_date)
ZFF = ZFF.drop('Adj Close', axis=1)
symbol = 'ZT=F'
ZTF = yf.download(symbol, start=start_date, end=end_date)
ZTF = ZTF.drop('Adj Close', axis=1)
symbol = 'GC=F'
GCF = yf.download(symbol, start=start_date, end=end_date)
GCF = GCF.drop('Adj Close', axis=1)
symbol = 'HG=F'
HGF = yf.download(symbol, start=start_date, end=end_date)
HGF = HGF.drop('Adj Close', axis=1)
symbol = 'SI=F'
SIF = yf.download(symbol, start=start_date, end=end_date)
SIF = SIF.drop('Adj Close', axis=1)
symbol = 'PL=F'
PLF = yf.download(symbol, start=start_date, end=end_date)
PLF = PLF.drop('Adj Close', axis=1)
CLF = yf.download(symbol, start=start_date, end=end_date)
CLF = CLF.drop('Adj Close', axis=1)
symbol = 'NG=F'
NGF = yf.download(symbol, start=start_date, end=end_date)
NGF = NGF.drop('Adj Close', axis=1)
symbol = 'BZ=F'
BZF = yf.download(symbol, start=start_date, end=end_date)
BZF = BZF.drop('Adj Close', axis=1)
symbol = 'ZC=F'
ZCF = yf.download(symbol, start=start_date, end=end_date)
ZCF = ZCF.drop('Adj Close', axis=1)
symbol = 'ZO=F'
ZOF = yf.download(symbol, start=start_date, end=end_date)
ZOF = ZOF.drop('Adj Close', axis=1)
symbol = 'KE=F'
KEF = yf.download(symbol, start=start_date, end=end_date)
KEF = KEF.drop('Adj Close', axis=1)
symbol = 'ZR=F'
ZRF = yf.download(symbol, start=start_date, end=end_date)
ZRF = ZRF.drop('Adj Close', axis=1)
symbol = 'ZM=F'
ZMF = yf.download(symbol, start=start_date, end=end_date)
ZMF = ZMF.drop('Adj Close', axis=1)
symbol = 'ZL=F'
ZLF = yf.download(symbol, start=start_date, end=end_date)
ZLF = ZLF.drop('Adj Close', axis=1)
symbol = 'ZS=F'
ZSF = yf.download(symbol, start=start_date, end=end_date)
ZSF = ZSF.drop('Adj Close', axis=1)
symbol = 'GF=F'
GFF = yf.download(symbol, start=start_date, end=end_date)
GFF = GFF.drop('Adj Close', axis=1)
symbol = 'HE=F'
HEF = yf.download(symbol, start=start_date, end=end_date)
HEF = HEF.drop('Adj Close', axis=1)
symbol = 'HO=F'
HOF = yf.download(symbol, start=start_date, end=end_date)
HOF = HOF.drop('Adj Close', axis=1)
symbol = 'LE=F'
LFF = yf.download(symbol, start=start_date, end=end_date)
LFF = LFF.drop('Adj Close', axis=1)
symbol = 'CC=F'
CCF = yf.download(symbol, start=start_date, end=end_date)
CCF = CCF.drop('Adj Close', axis=1)
symbol = 'KC=F'
KCF = yf.download(symbol, start=start_date, end=end_date)
KCF = KCF.drop('Adj Close', axis=1)
symbol = 'CT=F'
CTF = yf.download(symbol, start=start_date, end=end_date)
CTF = CTF.drop('Adj Close', axis=1)
symbol = 'OJ=F'
OJF = yf.download(symbol, start=start_date, end=end_date)
OJF = OJF.drop('Adj Close', axis=1)
symbol = 'SB=F'
SBF = yf.download(symbol, start=start_date, end=end_date)
SBF = SBF.drop('Adj Close', axis=1)


# EFTs

symbol = 'KBA'
KBA = yf.download(symbol, start=start_date, end=end_date)
KBA = KBA.drop('Adj Close', axis=1)
symbol = 'CHIQ'
CHIQ = yf.download(symbol, start=start_date, end=end_date)
CHIQ = CHIQ.drop('Adj Close', axis=1)
symbol = 'CNTX'
CNTX = yf.download(symbol, start=start_date, end=end_date)
CNTX = CNTX.drop('Adj Close', axis=1)
symbol = 'CHIS'
CHIS = yf.download(symbol, start=start_date, end=end_date)
CHIS = CHIS.drop('Adj Close', axis=1)
symbol = 'CNYA'
CNYA = yf.download(symbol, start=start_date, end=end_date)
CNYA = CNYA.drop('Adj Close', axis=1)
symbol = 'ASHX'
ASHX = yf.download(symbol, start=start_date, end=end_date)
ASHX = ASHX.drop('Adj Close', axis=1)
symbol = 'KFYP'
KFYP = yf.download(symbol, start=start_date, end=end_date)
KFYP = KFYP.drop('Adj Close', axis=1)
symbol = 'KGRN'
KGRN = yf.download(symbol, start=start_date, end=end_date)
KGRN = KGRN.drop('Adj Close', axis=1)
symbol = 'THD'
THD = yf.download(symbol, start=start_date, end=end_date)
THD = THD.drop('Adj Close', axis=1)
symbol = 'BBAX'
BBAX = yf.download(symbol, start=start_date, end=end_date)
BBAX = BBAX.drop('Adj Close', axis=1)
symbol = 'FEMS'
FEMS = yf.download(symbol, start=start_date, end=end_date)
FEMS = FEMS.drop('Adj Close', axis=1)
symbol = 'EZA'
EZA = yf.download(symbol, start=start_date, end=end_date)
EZA = EZA.drop('Adj Close', axis=1)
symbol = 'XSD'
XSD = yf.download(symbol, start=start_date, end=end_date)
XSD = XSD.drop('Adj Close', axis=1)
symbol = 'EYLD'
EYLD = yf.download(symbol, start=start_date, end=end_date)
EYLD = EYLD.drop('Adj Close', axis=1)
symbol = 'FNDE'
FNDE = yf.download(symbol, start=start_date, end=end_date)
FNDE = FNDE.drop('Adj Close', axis=1)
symbol = 'SPEM'
SPEM = yf.download(symbol, start=start_date, end=end_date)
SPEM = SPEM.drop('Adj Close', axis=1)
symbol = 'DXJS'
DXJS = yf.download(symbol, start=start_date, end=end_date)
DXJS = DXJS.drop('Adj Close', axis=1)
symbol = 'KURE'
KURE = yf.download(symbol, start=start_date, end=end_date)
KURE = KURE.drop('Adj Close', axis=1)
symbol = 'EWX'
EWX = yf.download(symbol, start=start_date, end=end_date)
EWX = EWX.drop('Adj Close', axis=1)
symbol = 'FLJH'
FLJH = yf.download(symbol, start=start_date, end=end_date)
FLJH = FLJH.drop('Adj Close', axis=1)
symbol = 'CQQQ'
CQQQ = yf.download(symbol, start=start_date, end=end_date)
CQQQ = CQQQ.drop('Adj Close', axis=1)
symbol = 'CHIE'
CHIE = yf.download(symbol, start=start_date, end=end_date)
CHIE = CHIE.drop('Adj Close', axis=1)
symbol = 'MFEM'
MFEM = yf.download(symbol, start=start_date, end=end_date)
MFEM = MFEM.drop('Adj Close', axis=1)
symbol = 'DGS'
DGS = yf.download(symbol, start=start_date, end=end_date)
DGS = DGS.drop('Adj Close', axis=1)
symbol = 'HEEM'
HEEM = yf.download(symbol, start=start_date, end=end_date)
HEEM = HEEM.drop('Adj Close', axis=1)


# World Composite Index


symbol = '^HSI'
HSI = yf.download(symbol, start=start_date, end=end_date)
HSI = HSI.drop('Adj Close', axis=1)
symbol = '000001.SS'
SSE = yf.download(symbol, start=start_date, end=end_date)
SSE = SSE.drop('Adj Close', axis=1)
symbol = '^N225'
N225 = yf.download(symbol, start=start_date, end=end_date)
N225 = N225.drop('Adj Close', axis=1)
symbol = '^KS11'
KS11 = yf.download(symbol, start=start_date, end=end_date)
KS11 = KS11.drop('Adj Close', axis=1)
symbol = '^BSESN'
BSESN = yf.download(symbol, start=start_date, end=end_date)
BSESN = BSESN.drop('Adj Close', axis=1)
symbol = '^MXX'
MXX = yf.download(symbol, start=start_date, end=end_date)
MXX = MXX.drop('Adj Close', axis=1)
symbol = '^TNX'
TNX = yf.download(symbol, start=start_date, end=end_date)
TNX = TNX.drop('Adj Close', axis=1)
symbol = '^VIX'
VIX = yf.download(symbol, start=start_date, end=end_date)
VIX = VIX.drop('Adj Close', axis=1)
symbol = '^BVSP'
BVSP = yf.download(symbol, start=start_date, end=end_date)
BVSP = BVSP.drop('Adj Close', axis=1)
symbol = '^IXIC'
IXIC = yf.download(symbol, start=start_date, end=end_date)
IXIC = IXIC.drop('Adj Close', axis=1)
symbol = '^GSPTSE'
GSPTSE = yf.download(symbol, start=start_date, end=end_date)
GSPTSE = GSPTSE.drop('Adj Close', axis=1)
symbol = '^DJI'
DJI = yf.download(symbol, start=start_date, end=end_date)
DJI = DJI.drop('Adj Close', axis=1)
symbol = '^FCHI'
FCHI = yf.download(symbol, start=start_date, end=end_date)
FCHI = FCHI.drop('Adj Close', axis=1)
symbol = '^GDAXI'
GDAXI = yf.download(symbol, start=start_date, end=end_date)
GDAXI = GDAXI.drop('Adj Close', axis=1)
symbol = '^FTSE'
FTSE = yf.download(symbol, start=start_date, end=end_date)
FTSE = FTSE.drop('Adj Close', axis=1)
symbol = '^IBEX'
IBEX = yf.download(symbol, start=start_date, end=end_date)
IBEX = IBEX.drop('Adj Close', axis=1)
symbol = '^N100'
N100 = yf.download(symbol, start=start_date, end=end_date)
N100 = N100.drop('Adj Close', axis=1)
symbol = '^GSPC'
GSPC = yf.download(symbol, start=start_date, end=end_date)
GSPC = GSPC.drop('Adj Close', axis=1)
symbol = '^RUT'
RUT = yf.download(symbol, start=start_date, end=end_date)
RUT = RUT.drop('Adj Close', axis=1)
symbol = '^NYA'
NYA = yf.download(symbol, start=start_date, end=end_date)
NYA = NYA.drop('Adj Close', axis=1)
symbol = '^STI'
STI = yf.download(symbol, start=start_date, end=end_date)
STI = STI.drop('Adj Close', axis=1)
symbol = '^AXJO'
AXJO = yf.download(symbol, start=start_date, end=end_date)
AXJO = AXJO.drop('Adj Close', axis=1)



In [ ]:
# test_inverse transformation
import csv
# train_data_list = [data3, data4, data5, data6, data7, data8, data9, data10
#                   , data11, data12, data13, data14, data15, data16, data17, data18, data19, data20]
# data21, data22, data23, data24, data25, data26, data27, data28, data29, data30, data31, data32, data33, data34, data35, data36, data37, data38, data39, data40
data = None
# train_data_list=[HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                  ESF,YMF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,HGF,SIF,NGF,BZF,ZCF,ZOF,KEF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                  SSE,N225,KS11,BSESN,FCHI,GDAXI,IBEX,DJI,GSPC,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO
#                 ]
# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,
#                  KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                  ESF,YMF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,HGF,SIF,NGF,BZF,ZCF,ZOF,KEF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                  SSE,N225,KS11,BSESN,FCHI,GDAXI,IBEX,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO
#                 ]
# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,
#                 MSFT,AAPL,GOOGL,GOOG,AMZN,META,AMD,ASML,NVDA,TSM,TSLA,
#                 SSE,N225,KS11,BSESN,FCHI,GDAXI,IBEX,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO
#                 ]
# train_data_list=[ESF,YMF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,HGF,SIF,NGF,BZF,ZCF,ZOF,KEF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                 ]
# train_data_list=[ESF,YMF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,HGF,SIF,NGF,BZF,ZCF,ZOF,KEF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                 ]
# train_data_list=[ESF,YMF,NQF,RTYF,ZBF,ZNF,ZFF
#                 ]
train_data_list = [MSFT,AAPL,GOOGL]
# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,
#                  FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,GSPC,DJI,RUT,
#                  HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,ESF,YMF,NQF,
#                  RTYF,ZBF,ZNF,CLF,GCF,HGF,SIF,CLF,NGF,ZCF,KEF,MSFT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,TSLA,AMD,ASML]

# valid_data_list=[SSE,N225,KS11,BSESN,FCHI,GDAXI,IBEX,DJI,GSPC,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
# valid_data_list=[WMT,TGT,COST,HD,LOW,PG,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,FDX,UPS]

# train_data_list=[ESF,YMF,NQF,RTYF,ZBF,ZNF,CLF,GCF,HGF,SIF,CLF,NGF,ZCF,ZFF,ZTF,PLF,PAF,BZF,ZOF,KCF,CTF]
# train_data_list=[ESF,YMF,NQF,ZBF,ZNF,GCF,HGF,SIF,CLF,ZCF,NGF,WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,CVS,KO,PEP]

valid_data_list = [JNJ]


symbol='JNJ'
start_date = '2021-01-01'
end_date = '2023-10-08'
start_date_short = '2023-01-01'
end_date_short = '2023-10-08'
target_col = 'Close'

n_trials = 20
n_top_models = 1
n_predict = 5
n_last_sequence = 100
forward= 0


save_directory = "/content/drive/MyDrive/Colab Notebooks/loaded_model/y_seq_len/jnj_1"
# save_directory = "/content/drive/MyDrive/Colab Notebooks/Fixed_Model/Indicators/merge_seq/gspc/gspc_1"
all_results_metrics_df, all_future_predictions_df, all_future_metrics_df = random_search(
    data=data, train_data_list=train_data_list, valid_data_list=valid_data_list, symbol=symbol, start_date=start_date, end_date=end_date,
    start_date_short=start_date_short, end_date_short=end_date_short, target_col=target_col, n_trials=n_trials, n_top_models=n_top_models,
    model_save=True, save_directory=save_directory, plot_loss=False, predict_plot=True, future_plot=False, overall_future_plot=True,
    use_target_col=False, future_predictions=None, n_last_sequence=n_last_sequence, forward=forward)

# results_df, all_future_predictions_df, all_future_metrics_df = random_search(
#     data=data, train_data_list=train_data_list, valid_data_list=valid_data_list, symbol=symbol, start_date=start_date, end_date=end_date,
#     start_date_short=start_date_short, end_date_short=end_date_short, target_col=target_col, n_trials=n_trials, n_top_models=n_top_models,
#     model_save=True, save_directory=save_directory, plot_loss=False, predict_plot=True, future_plot=False, future_plot=True,
#     use_target_col=False, future_predictions=None, n_predict=n_predict, n_last_sequence=n_last_sequence, forward=forward)


output_file_path = "/content/drive/MyDrive/Colab Notebooks/loaded_model/y_seq_len/jnj_1"
# Save results_df
all_results_metrics_df.to_csv(f'{output_file_path}_all_results_metrics.csv', index=True)

# #Save top_models
# top_models_df = pd.DataFrame(top_models, columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
# top_models_df.to_csv(f'{output_file_path}_top_models.csv', index=True)

# Save the future_metrics DataFrame to a CSV file
# all_future_metric_finals.to_csv(f'{output_file_path}_all_future_metrics.csv', index=True)
# Save future_predictions
all_future_predictions_df.to_csv(f'{output_file_path}_all_future_predictions.csv', index=True)
# Save the overall_future_metrics DataFrame to a CSV file
all_future_metrics_df.to_csv(f'{output_file_path}_all_future_metrics.csv', index=True)